### 基础线程

In [1]:
import threading
import time

# 线程函数
def worker(name):
    print(f"{name} 开始工作")
    time.sleep(2)  # 模拟耗时
    print(f"{name} 工作完成")

# 创建线程
t1 = threading.Thread(target=worker, args=("线程1",))
t2 = threading.Thread(target=worker, args=("线程2",))

# 启动线程
t1.start()
t2.start()

# 等待线程结束
t1.join()
t2.join()

print("所有线程完成")

线程1 开始工作
线程2 开始工作
线程2 工作完成
线程1 工作完成
所有线程完成


### 类方式创建线程

In [2]:
import threading

class MyThread(threading.Thread):
    def __init__(self, name):
        super().__init__()
        self.name = name
    
    def run(self):
        print(f"{self.name} 运行中...")
        for i in range(3):
            print(f"{self.name}: {i}")
            time.sleep(0.5)

# 使用
t1 = MyThread("线程A")
t2 = MyThread("线程B")

t1.start()
t2.start()

t1.join()
t2.join()

线程A 运行中...
线程A: 0
线程B 运行中...
线程B: 0
线程A: 1线程B: 1

线程A: 2线程B: 2



### 线程锁

In [3]:
import threading

counter = 0
lock = threading.Lock()  # 创建锁

def increment():
    global counter
    for _ in range(100000):
        with lock:  # 自动加锁/解锁
            counter += 1
        # 等价于：
        # lock.acquire()
        # counter += 1
        # lock.release()

# 创建线程
threads = []
for i in range(5):
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

# 等待所有线程
for t in threads:
    t.join()

print(f"最终计数: {counter}")  # 应该是 500000

最终计数: 500000


###  线程通信

In [4]:
import threading
import queue

# 生产者-消费者模式
def producer(q):
    for i in range(5):
        item = f"产品{i}"
        q.put(item)
        print(f"生产: {item}")
        time.sleep(0.5)
    q.put(None)  # 结束信号

def consumer(q):
    while True:
        item = q.get()
        if item is None:  # 收到结束信号
            break
        print(f"消费: {item}")
        time.sleep(1)

# 创建队列
q = queue.Queue()

# 创建线程
p = threading.Thread(target=producer, args=(q,))
c = threading.Thread(target=consumer, args=(q,))

p.start()
c.start()

p.join()
c.join()
print("生产消费完成")

生产: 产品0
消费: 产品0
生产: 产品1
消费: 产品1生产: 产品2

生产: 产品3
消费: 产品2
生产: 产品4
消费: 产品3
消费: 产品4
生产消费完成


### 线程池

In [5]:
from concurrent.futures import ThreadPoolExecutor
import time

def task(name, duration):
    print(f"{name} 开始")
    time.sleep(duration)
    return f"{name} 完成"

# 创建线程池（最多3个线程）
with ThreadPoolExecutor(max_workers=3) as executor:
    # 提交任务
    futures = [
        executor.submit(task, f"任务{i}", i)
        for i in range(1, 6)
    ]
    
    # 获取结果
    for future in futures:
        print(future.result())

任务1 开始
任务2 开始
任务3 开始
任务4 开始任务1 完成

任务5 开始
任务2 完成
任务3 完成
任务4 完成
任务5 完成


### 注意事项

#### GIL限制

In [7]:
# Python有GIL（全局解释器锁）
# 对于CPU密集型任务，多线程可能不会提高速度
# 对于I/O密集型任务（网络、磁盘），多线程有效

# CPU密集型 - 效果有限
def cpu_task(n):
    for _ in range(n):
        _ = 1 + 1

# I/O密集型 - 效果明显  
def io_task():
    time.sleep(1)  # 模拟I/O等待

#### 守护线程

In [ ]:
def daemon_worker():
    while True:
        print("守护线程运行中...")
        time.sleep(1)

# 创建守护线程
daemon = threading.Thread(target=daemon_worker, daemon=True)
daemon.start()

# 主线程结束后，守护线程自动结束
time.sleep(3)
print("主线程结束")

#### 线程局部数据

In [1]:
import threading

# 每个线程独立的数据
local_data = threading.local()

def show_data():
    print(f"线程 {threading.current_thread().name}: {local_data.value}")

def worker(value):
    local_data.value = value
    show_data()

# 每个线程有自己的local_data.value
threads = []
for i in range(3):
    t = threading.Thread(target=worker, args=(f"数据{i}",), name=f"线程{i}")
    threads.append(t)
    t.start()

for t in threads:
    t.join()

线程 线程0: 数据0
线程 线程1: 数据1
线程 线程2: 数据2


### 实际案例

#### 下载器

In [ ]:
import threading
import requests

def download_file(url, filename):
    """下载文件"""
    print(f"开始下载: {filename}")
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"下载完成: {filename}")

# 要下载的文件列表
files = [
    ("https://example.com/file1.txt", "file1.txt"),
    ("https://example.com/file2.txt", "file2.txt"),
    ("https://example.com/file3.txt", "file3.txt")
]

# 创建下载线程
threads = []
for url, filename in files:
    t = threading.Thread(target=download_file, args=(url, filename))
    threads.append(t)
    t.start()

# 等待所有下载完成
for t in threads:
    t.join()

print("所有文件下载完成")

#### 网站监控

In [ ]:
import threading
import requests
import time

def check_website(url):
    """检查网站状态"""
    try:
        response = requests.get(url, timeout=5)
        status = "正常" if response.status_code == 200 else "异常"
        print(f"{url}: {status}")
    except:
        print(f"{url}: 无法访问")

websites = [
    "https://www.baidu.com",
    "https://www.google.com",
    "https://www.github.com",
    "https://www.python.org"
]

def monitor():
    while True:
        print(f"\n检查时间: {time.strftime('%H:%M:%S')}")
        
        # 为每个网站创建线程
        threads = []
        for url in websites:
            t = threading.Thread(target=check_website, args=(url,))
            threads.append(t)
            t.start()
        
        # 等待所有检查完成
        for t in threads:
            t.join()
        
        time.sleep(60)  # 每分钟检查一次

# monitor()  # 取消注释运行

#### 计算密集型任务

In [6]:
import threading
import math

def calculate_prime(start, end, result_list):
    """计算区间内的质数"""
    primes = []
    for num in range(start, end + 1):
        if num > 1:
            for i in range(2, int(math.sqrt(num)) + 1):
                if num % i == 0:
                    break
            else:
                primes.append(num)
    result_list.extend(primes)

# 分段计算
result = []
threads = []
n = 1000000
num_threads = 4
chunk_size = n // num_threads

for i in range(num_threads):
    start = i * chunk_size + 1
    end = (i + 1) * chunk_size if i < num_threads - 1 else n
    t = threading.Thread(target=calculate_prime, args=(start, end, result))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print(f"找到 {len(result)} 个质数")

找到 78498 个质数
